# Webshell Detection With Command Line Keywords
Detects certain command line parameters often used during reconnaissance activity via web shells

## Rule Content
```
- title: Webshell Detection With Command Line Keywords
  id: bed2a484-9348-4143-8a8a-b801c979301c
  description: Detects certain command line parameters often used during reconnaissance
    activity via web shells
  author: Florian Roth
  reference:
  - https://www.fireeye.com/blog/threat-research/2013/08/breaking-down-the-china-chopper-web-shell-part-ii.html
  date: 2017/01/01
  modified: 2019/10/26
  tags:
  - attack.privilege_escalation
  - attack.persistence
  - attack.t1100
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      ParentImage:
      - '*\apache*'
      - '*\tomcat*'
      - '*\w3wp.exe'
      - '*\php-cgi.exe'
      - '*\nginx.exe'
      - '*\httpd.exe'
      CommandLine:
      - '*whoami*'
      - '*net user *'
      - '*ping -n *'
      - '*systeminfo'
      - '*&cd&echo*'
      - '*cd /d*'
    condition: selection
  fields:
  - CommandLine
  - ParentCommandLine
  falsepositives:
  - unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_parent_path.keyword:(*\\apache* OR *\\tomcat* OR *\\w3wp.exe OR *\\php\-cgi.exe OR *\\nginx.exe OR *\\httpd.exe) AND process_command_line.keyword:(*whoami* OR *net\ user\ * OR *ping\ \-n\ * OR *systeminfo OR *&cd&echo* OR *cd\ \/d*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()